# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [30]:
%pip install sentence-transformers
%pip install faiss-cpu
%pip install openai


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [31]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [32]:
K = 25

In [33]:
df_news = pd.read_csv('df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [34]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [35]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.
 

In [36]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'], 
        'TICKER': row['TICKER'], 
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [37]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [38]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [39]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [40]:
#IMPORT AND INITIALIZE OPENAI CLIENT

from openai import OpenAI
from sk import my_sk   # IMPORT SECRET KEY FROM sk.py FILE

client = OpenAI(api_key=my_sk)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Write a one-sentence bedtime story about a unicorn."}]
)

print(response.choices[0].message.content)

As the moonlight shimmered through the enchanted forest, Luna the unicorn twirled with joy, knowing that her dreams of spreading magic would soon come true, bringing happiness to all who believed.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [41]:
import yfinance as yf
import time

# TICKERS FROM NEWS DATASET
tickers = df_news['TICKER'].dropna().unique().tolist()
meta_data = []

for ticker in tickers:
    try:
        company = yf.Ticker(ticker)
        info = company.info

        meta_data.append({
            'TICKER': ticker,
            'COMPANY_NAME': info.get('longName', 'N/A'),
            'SECTOR': info.get('sector', 'N/A'),
            'INDUSTRY': info.get('industry', 'N/A')
        })
    except Exception as e:
        print(f"[ERROR] {ticker}: {e}")
        meta_data.append({
            'TICKER': ticker,
            'COMPANY_NAME': 'N/A',
            'SECTOR': 'N/A',
            'INDUSTRY': 'N/A'
        })

    # FOR GOOOD LUCK
    time.sleep(0.1)

df_meta = pd.DataFrame(meta_data)
display(df_meta.head())

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [42]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

In [43]:
#MERGE DF_NEWS WITH DF_META
#df_news = df_news.merge(df_meta, on='TICKER', how='left')

In [44]:
#RAG (TAKES THE USER QUESTION AS INPUT)

def rag_query_pipeline(question, vector_store, client, k=5, model="gpt-4o-mini"):
    
    #Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
    retrieved = vector_store.search(question, k=k)

    if not retrieved:
        return "There isn't any relevant info.", []

    # Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
    # SNIPPET COMBINE FOR CONTEXT
    context_snippets = [doc for doc, _, _ in retrieved]
    context_text = "\n\n".join(f"- {snippet}" for snippet in context_snippets)

    # PROMPT
    prompt = f"""You are a financial analyst AI. Based on the following news snippets, answer the user's question.

### News Context:
{context_text}

### Question:
{question}

### Answer (short and precise):"""

    # PROMPT TO AI
    response = client.chat.completions.create(model=model,messages=[{"role": "user", "content": prompt}])

    answer = response.choices[0].message.content

    # Returns both the generated answer and the underlying news snippets used for context.
    print_markdown(f"###**Generated Answer**:\n{answer}\n")
    print_markdown(f"---\n###**News Context Used**:\n{context_text}")

    return answer, context_snippets

#### Predefined questions

In [45]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [46]:
# FUNCTION TO RESPOND QUERY
def run_rag_batch(questions, category, vector_store, client, k=5):
    print_markdown(f"#Category: **{category}**")

    for i, question in enumerate(questions, 1):
        print_markdown(f"---\n##  Q{i}: {question}")
        rag_query_pipeline(question, vector_store, client, k=k)

# Ejecutar cada categoría
run_rag_batch(questions_topic, "Topic-based Questions", faiss_store, client)
run_rag_batch(questions_company, "Company-based Questions", faiss_store, client)
run_rag_batch(questions_industry, "Industry-based Questions", faiss_store, client)

#Category: **Topic-based Questions**

---
##  Q1: What are the major concerns expressed in financial news about inflation?

###**Generated Answer**:
The major concerns regarding inflation highlighted in the financial news include persistent US inflation risks as indicated by the Federal Reserve's May policy meeting, and rising food inflation which is dampening hopes for a rate cut amid ongoing tariff issues.


---
###**News Context Used**:
- Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

---
##  Q2: How is investor sentiment described in recent financial headlines?

...

###**Generated Answer**:
The retail industry is experiencing volatility in demand and has seen a substantial decline in stock performance, with retail stocks tumbling by 13.7% over the past six months, worse than the S&P 500’s 5.5% loss. Retailers are adjusting their business models in response to technological changes and shifting supply chains due to tariffs. Overall, consumer discretionary businesses are also feeling the impact of economic cycles, with a reported industry drop of 12.3%.


---
###**News Context Used**:
- 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.

- Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.

- 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.

- Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...

- Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

---
##  Q3: What risks or challenges are discussed in recent news about the energy industry?

###**Generated Answer**:
Recent news highlights several risks and challenges in the energy industry, particularly for renewable energy stocks facing potential decimation due to a U.S. Congressional bill that could repeal crucial subsidies. Additionally, oilfield service companies like SLB, HAL, and BKR are grappling with declining oil prices, rising tariffs, and shrinking drilling budgets, raising concerns about their future viability.


---
###**News Context Used**:
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

- 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance** 
2. **Completeness**  
3. **Bias or Noise** 
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

_The news snippets does support the generated answers, the key facts and themes on the context made its way to the anser generated by the LLM model, also, the generated answer its like a resume of the top news snippets retrieved by the FAISS vector store_


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

_For me the generated answers fully addres certain types of questions, when the question its about something specifis, the generated answer fully addresses the question, however the sentiment like questions pose a greater challenge, specifically in the question_

_"How is investor sentiment described in recent financial headlines?"_


_It is one of the most dificult questions, because the model has to make a consensus with the information of the snippets retrived via FAISS vector store, and because of the broader range of opinions in the news, the generated answer its some vague_

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

_There are some irrelevant news retrieves by the FAISS vector store, and this could be attributed to the quality of the embedding model, a larger model creates a more contextualy richer embeddeing, wich by the way, allows to obtain  a more semantic similar results"_

_In the question Q2: What trends are being reported in the retail industry?_

_One retrieved news snippet was this:_

_"3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline."_

_However Consumer Discretionary and Retail Industry are related, but they arent the same, so in this case this could affect the quality of the output in form of noise_

#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

_The snippets aparently didn´t show conflicting viewpoints, and the information in the most part was consistent, however, the nature of the stock market it's about uncertainity, so, for some questions where it has to be a consensus, it will be a little bit tricky to the LLM.

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

_The way the retrieval and the generation process could be improved by bruteforce, since we have analyzing the news wiith an AI tools, the more data we analyze, the better undestanding of the problem we got, also, when we incorporate more infromation, they can appear underlying factors or insights._

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [47]:
#WE HAVE TO MERGE DF_NEWS DATAFRAME AND DF_META

df_news_merged = df_news.merge(df_meta, on='TICKER', how='left')
display(df_news_merged)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL,EMBEDDED_TEXT,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...,2 Dow Jones Stocks with Promising Prospects an...,3M Company,Industrials,Conglomerates
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...,3 S&P 500 Stocks Skating on Thin Ice : The S&P...,3M Company,Industrials,Conglomerates
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...,3M Rises 15.8% YTD: Should You Buy the Stock N...,3M Company,Industrials,Conglomerates
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,3M Company,Industrials,Conglomerates
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...,3 Cash-Producing Stocks with Questionable Fund...,3M Company,Industrials,Conglomerates
...,...,...,...,...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis Inc.,Healthcare,Drug Manufacturers - Specialty & Generic
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis Inc.,Healthcare,Drug Manufacturers - Specialty & Generic
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,Zoetis Inc.,Healthcare,Drug Manufacturers - Specialty & Generic
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...,Zoetis (ZTS) Upgraded to Buy: Here's Why : Zoe...,Zoetis Inc.,Healthcare,Drug Manufacturers - Specialty & Generic


In [48]:
def rag_meta_query_pipeline(
    question,
    vector_store,
    client,
    df_meta,
    k=5,
    model="gpt-4o-mini",
    debug=False,              
    truncate_chars=None       #TRUNCATE
):
    retrieved = vector_store.search(question, k=k)
    if not retrieved:
        if debug:
            print_markdown("**[DEBUG]** No retrieved results.")
        return "There isn't any relevant info.", []

    enriched_snippets = []
    used_docs = []

    for doc, metadata, score in retrieved:  
        if not isinstance(metadata, dict):
            metadata = {}

        # Recortar snippet si se pide
        doc_out = doc if truncate_chars is None else (doc[:truncate_chars] + "…") if len(doc) > truncate_chars else doc

        ticker = metadata.get("TICKER", None)

        meta_row = None
        if ticker and ticker in df_meta["TICKER"].values:
            meta_row = df_meta[df_meta["TICKER"] == ticker].iloc[0]

        if meta_row is not None:
            company  = meta_row.get("COMPANY_NAME", "N/A")
            sector   = meta_row.get("SECTOR", "N/A")
            industry = meta_row.get("INDUSTRY", "N/A")
            enriched_text = f"""Ticker: {ticker}
Company: {company}
Sector: {sector}
Industry: {industry}
News: {doc_out}"""
        else:
            enriched_text = f"""Ticker: {ticker if ticker else "N/A"}
News: {doc_out}"""

        enriched_snippets.append(enriched_text)
        used_docs.append({"doc": doc, "metadata": metadata, "score": score})

    context_text = "\n\n".join(f"- {s}" for s in enriched_snippets)

    prompt = f"""You are a financial analyst AI. Based on the following news snippets and their associated metadata, answer the user's question.

### News Context (with metadata):
{context_text}

### Question:
{question}

### Answer (short and precise):"""

    # DEBUG
    if debug:
        print_markdown(
            f"<details><summary><strong>DEBUG · Prompt enviado al modelo</strong></summary>\n\n"
            f"Tokens aprox: {len(prompt.split())}\n\n"
            f"```text\n{prompt}\n```\n"
            f"</details>"
        )

    resp = client.chat.completions.create(model=model, messages=[{"role":"user","content":prompt}])
    answer = resp.choices[0].message.content

    print_markdown(f"### **Generated Answer**:\n{answer}\n")
    print_markdown(f"---\n### **News Context Used**:\n{context_text}")

    return answer, used_docs


def run_rag_batch(questions, category, vector_store, client, df_meta, k=5, debug_prompts=False, truncate_chars=None):
    print_markdown(f"# Category: **{category}**")
    for i, question in enumerate(questions, 1):
        print_markdown(f"---\n## Q{i}: {question}")
        rag_meta_query_pipeline(
            question,
            vector_store,
            client,
            df_meta,
            k=k,
            debug=debug_prompts,       
            truncate_chars=truncate_chars
        )

run_rag_batch(questions_topic, "Topic-based Questions", faiss_store, client, df_meta, debug_prompts=True)
run_rag_batch(questions_company, "Company-based Questions", faiss_store, client, df_meta, debug_prompts=True)
run_rag_batch(questions_industry, "Industry-based Questions", faiss_store, client, df_meta, debug_prompts=True)

# Category: **Topic-based Questions**

---
## Q1: What are the major concerns expressed in financial news about inflation?

<details><summary><strong>DEBUG · Prompt enviado al modelo</strong></summary>

Tokens aprox: 259

```text
You are a financial analyst AI. Based on the following news snippets and their associated metadata, answer the user's question.

### News Context (with metadata):
- Ticker: BLK
Company: BlackRock, Inc.
Sector: Financial Services
Industry: Asset Management
News: Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

- Ticker: TSLA
Company: Tesla, Inc.
Sector: Consumer Cyclical
Industry: Auto Manufacturers
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- Ticker: NVDA
Company: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- Ticker: LULU
Company: lululemon athletica inc.
Sector: Consumer Cyclical
Industry: Apparel Retail
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- Ticker: AVGO
Company: Broadcom Inc.
Sector: Technology
Industry: Semiconductors
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

### Question:
What are the major concerns expressed in financial news about inflation?

### Answer (short and precise):
```
</details>

### **Generated Answer**:
The major concerns expressed in financial news about inflation include persistent US inflation risks highlighted by the Federal Reserve, which may hinder prospects for interest rate cuts. Additionally, food inflation is dampening hopes of economic recovery, indicating broader implications for consumer spending and economic growth.


---
### **News Context Used**:
- Ticker: BLK
Company: BlackRock, Inc.
Sector: Financial Services
Industry: Asset Management
News: Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

- Ticker: TSLA
Company: Tesla, Inc.
Sector: Consumer Cyclical
Industry: Auto Manufacturers
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- Ticker: NVDA
Company: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- Ticker: LULU
Company: lululemon athletica inc.
Sector: Consumer Cyclical
Industry: Apparel Retail
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

- Ticker: AVGO
Company: Broadcom Inc.
Sector: Technology
Industry: Semiconductors
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

...

---
### **News Context Used**:
- Ticker: KMX
Company: CarMax, Inc.
Sector: Consumer Cyclical
Industry: Auto & Truck Dealerships
News: 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.

- Ticker: BBY
Company: Best Buy Co., Inc.
Sector: Consumer Cyclical
Industry: Specialty Retail
News: Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.

- Ticker: HLT
Company: Hilton Worldwide Holdings Inc.
Sector: Consumer Cyclical
Industry: Lodging
News: 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.

- Ticker: PKG
Company: Packaging Corporation of America
Sector: Consumer Cyclical
Industry: Packaging & Containers
News: Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...

- Ticker: APD
Company: Air Products and Chemicals, Inc.
Sector: Basic Materials
Industry: Specialty Chemicals
News: Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

---
## Q3: What risks or challenges are discussed in recent news about the energy industry?

<details><summary><strong>DEBUG · Prompt enviado al modelo</strong></summary>

Tokens aprox: 446

```text
You are a financial analyst AI. Based on the following news snippets and their associated metadata, answer the user's question.

### News Context (with metadata):
- Ticker: NEE
Company: NextEra Energy, Inc.
Sector: Utilities
Industry: Utilities - Regulated Electric
News: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

- Ticker: ENPH
Company: Enphase Energy, Inc.
Sector: Technology
Industry: Solar
News: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

- Ticker: HAL
Company: Halliburton Company
Sector: Energy
Industry: Oil & Gas Equipment & Services
News: Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

- Ticker: BKR
Company: Baker Hughes Company
Sector: Energy
Industry: Oil & Gas Equipment & Services
News: Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

- Ticker: FCX
Company: Freeport-McMoRan Inc.
Sector: Basic Materials
Industry: Copper
News: 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

### Question:
What risks or challenges are discussed in recent news about the energy industry?

### Answer (short and precise):
```
</details>

### **Generated Answer**:
Recent news highlights several risks and challenges in the energy industry, particularly for renewable energy stocks, including the potential repeal of crucial subsidies by the U.S. Congress, which could render many renewable projects uneconomical. Additionally, the oil and gas sector faces pressures from sliding oil prices, rising tariffs, and shrinking drilling budgets, raising concerns about the future of oilfield services.


---
### **News Context Used**:
- Ticker: NEE
Company: NextEra Energy, Inc.
Sector: Utilities
Industry: Utilities - Regulated Electric
News: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

- Ticker: ENPH
Company: Enphase Energy, Inc.
Sector: Technology
Industry: Solar
News: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

- Ticker: HAL
Company: Halliburton Company
Sector: Energy
Industry: Oil & Gas Equipment & Services
News: Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

- Ticker: BKR
Company: Baker Hughes Company
Sector: Energy
Industry: Oil & Gas Equipment & Services
News: Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

- Ticker: FCX
Company: Freeport-McMoRan Inc.
Sector: Basic Materials
Industry: Copper
News: 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



### TOPIC


"What are the major concerns expressed in financial news about inflation?",

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [to be fair, the text is clear]     | [the text its also clear]   |
| Detail & Depth         | [gives more detail]     | [its more consice]   |
| Use of Context        | [good use of the context]     | [good use of the context]   |
| Accuracy & Grounding       | [the answer its accurate]     | [the answer its accurate]   |
| Relevance      | [snippets are relevent to the answer]     | [snippets are relevent to the answer]   |
| Narrative Flow      | [the answer could be condensed]     | [Great answer, really concise]   |

"How is investor sentiment described in recent financial headlines?",

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [it lacks a little bit of clarity]     | [the test its clear]   |
| Detail & Depth         | [brings detail but it didn´t answer the question]     | [the answer is concise and correct]   |
| Use of Context        | [good use of context]     | [good use of context]   |
| Accuracy & Grounding       | [it lacks accuracy]     | [it´s more accurate]   |
| Relevance      | [the snippets are relevant, but the answer don´t]     | [the snippets were relevant, as well as the answer]   |
| Narrative Flow      | [it didnt condense the information]     | [the answer was spot on]   |

"What role is artificial intelligence playing in recent finance-related news stories?"

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [its more like a resume of the snippets]     | [Great clarity]   |
| Detail & Depth         | [the answer its detailed enough]     | [the analysis it's more detailed]   |
| Use of Context        | [the answer make good use of context]     | []   |
| Accuracy & Grounding       | [its acurrate]     | [its accurate]   |
| Relevance      | [the snippets are somewaht relevant]     | [the snippets and the anwer are more relevant]   |
| Narrative Flow      | [its like a resumme in general]     | [the metadata made the anwer generated more useful]   |


### COMPANY

"How is Microsoft being portrayed in news stories about artificial intelligence?",

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [n/a]     | [n/a]   |
| Detail & Depth         | [n/a]     | [n/a]   |
| Use of Context        | [n/a]     | [n/a]   |
| Accuracy & Grounding       | [n/a]     | [n/a]   |
| Relevance      | [n/a]     | [n/a]   |
| Narrative Flow      | [n/a]     | [n/a]   |

NOTE: There are news of Microsoft related to AI, however in both versions of the pipeline the didn make it way to the top snippets.


"What financial news headlines connect Amazon with automation or logistics?"

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [the generated answer its clear]     | [the anser is a little more consice]   |
| Detail & Depth         | [the answer its detailed]     | [the anser is similar]   |
| Use of Context        | [good use of context]     | [the anser employs similar conext]   |
| Accuracy & Grounding       | [the anwer its accurate]     | [the answer its accurate] 
| Relevance      | [the snippets are relevant to the question]     | [the answer its relevant]   |
| Narrative Flow      | [it only presents the 2 news required]     | [the anser is similar]   |


### INDUSTRY

"What are the main themes emerging in financial news about the semiconductor industry?",

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [the answer is somewhat clear]     | [the anser is more concise]   |
| Detail & Depth         | [it lackss detail and depth]     | [the anser has some detail]   |
| Use of Context        | [the snippets are biassed]     | [the snippets are a little less biased]   |
| Accuracy & Grounding       | [the news are biassed toward one company]     | [the anser is a biassed]   |
| Relevance      | [the answer is somwhat relevant]     | [the anser is somewhat relevant]   |
| Narrative Flow      | [it lacks narrative flow, the answer are bullet points]     | [it has good narrative flow]   |


"What trends are being reported in the retail industry?",

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [the answer is clear]     | [the anser is similar]   |
| Detail & Depth         | [the answer has a little detail]     | [the anser is similar]   |
| Use of Context        | [the snippets are relevant]     | [the anser is similar]   
| Accuracy & Grounding       | [it has a degre of accuracy]     | [the anser is similar]   |
| Relevance      | [globaly the answer is relevant]     | [the anser is similar]   |
| Narrative Flow      | [has goog narrative flow]     | [the anser is similar]   |


"What risks or challenges are discussed in recent news about the energy industry?"

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [it lacks clarity]     | [the anser its very clear]   |
| Detail & Depth         | [the answer has little datil]     | [the anser has enough detail]   |
| Use of Context        | [the answer loks like a poor sumary]     | [the answer make great use of snippets]   |
| Accuracy & Grounding       | [the answer is somewhat accurate]     | [the answer its accurate]   |
| Relevance      | [the answer its somewhat relevant]     | [the answer is relevant to the question]   |
| Narrative Flow      | [the narrative flow is basic]     | [it has great narrative flow]   |



### CONSLUSION

_THE RAG METHOD IMPROVES THE RELEVANT INFO WE CAN GET FROM THE NEWS OUTLET, ITS IMPORTANT TO IMPROVE THE METHOD, HERE IN THE EXCERCISE WE GET BETTER RESULTS WHEN WE ADD METADATA TO THE LLM PROMPT, HOWEVER THE ORDER OF THE SNIPPETS REALLY AFFECT THE FINAL RESULTS._